In [1]:
%matplotlib notebook
from rfsoc_rfdc.rfsoc_overlay import RFSoCOverlay
from rfsoc_rfdc.overlay_task import OverlayTask
from rfsoc_rfdc.overlay_task import BlinkLedTask

from rfsoc_rfdc.transmitter.single_ch_transmitter_task import SingleChTransmitterTask
from rfsoc_rfdc.receiver.single_ch_receiver_task import SingleChReceiverTask

from rfsoc_rfdc.transmitter.multi_ch_transmitter_task import MultiChTransmitterTask
from rfsoc_rfdc.receiver.multi_ch_receiver_task import MultiChReceiverTask

from rfsoc_rfdc.rfdc_task import RfdcTask 
from rfsoc_rfdc.cmac_task import CmacTask

ol = RFSoCOverlay()

2024-05-09 17:18:03,927 - root - INFO - "Writing bitstream rfsoc_rfdc_v27.bit to RFSoC."


axi_gpio_led
axi_rfdc_intc
usp_rf_data_converter
dac_datapath/t230/axi_dma
dac_datapath/t230/fifo_count
adc_datapath/t226/adc_packet_generator
adc_datapath/t226/axi_dma
adc_datapath/t226/fifo_count
zynq_ultra_ps_e


In [ ]:
import sys
import os

def validate_arguments(waveform_path, dac_samp_rate, adc_samp_rate, carrier_freq):
    if not os.path.exists(waveform_path):
        raise FileNotFoundError(f"The specified waveform file '{waveform_path}' does not exist.")
    try:
        dac_samp_rate, adc_samp_rate = int(dac_samp_rate), int(adc_samp_rate)
        carrier_freq = int(carrier_freq)
    except ValueError:
        raise ValueError("DAC/ADC sampling rate and carrier frequency must be integers.")
    if dac_samp_rate < 500e6 or dac_samp_rate > 10e9:
        raise ValueError("DAC sampling rate ranges from 500MHz to 10GHz.")
    if adc_samp_rate < 500e6 or adc_samp_rate > 2.5e9:
        raise ValueError("DAC sampling rate ranges from 500MHz to 2.5GHz.")
    return True

matlab_waveform = "./wave_files/Tx.mat"
dac_samp_rate, adc_samp_rate = 2e9, 2e9
carrier_freq = 0

try:
    if validate_arguments(matlab_waveform, dac_samp_rate, adc_samp_rate, carrier_freq):
        print("Arguments validated successfully.")
except Exception as e:
    print(f"Error: {e}")
    sys.exit(1)

rfdc_t = RfdcTask(ol, dac_samp_rate, adc_samp_rate, carrier_freq)
# cmac_t = CmacTask(ol)
led_t = BlinkLedTask(ol)
tx_t = SingleChTransmitterTask(ol, matlab_waveform)
# tx_t = MultiChTransmitterTask(ol)
rx_t = SingleChReceiverTask(ol)
# rx_t = MultiChReceiverTask(ol)

for task in [rfdc_t]:
    task.start()
    task.join()

parallel_task = [led_t, tx_t, rx_t]
for task in parallel_task:
    task.start()
for task in parallel_task:
    task.join()

Arguments validated successfully.


FigureWidget({
    'data': [],
    'layout': {'template': '...',
               'title': {'text': 'Complex Sig…

FigureWidget({
    'data': [{'name': 'Magnitude', 'type': 'scattergl', 'uid': '31648392-e4b5-47a6-b8b2-5acb8dd…

2024-05-09 17:18:07,798 - root - INFO - Configuring LMK chip using file LMK04828_250.0.txt at 250.0 MHz
2024-05-09 17:18:07,804 - root - INFO - Configuring LMX chip using file LMX2594_500.0.txt at 500.0 MHz
2024-05-09 17:18:10,255 - root - INFO - DAC tile 1 (229) is fully powered up!
2024-05-09 17:18:10,313 - root - INFO - DAC tile 1 DAC block 0 is enabled!
2024-05-09 17:18:10,315 - root - INFO - DAC tile 1 DAC block 1 is NOT enabled!
2024-05-09 17:18:10,320 - root - INFO - DAC tile 1 DAC block 2 is NOT enabled!
2024-05-09 17:18:10,323 - root - INFO - DAC tile 1 DAC block 3 is NOT enabled!
2024-05-09 17:18:11,372 - root - INFO - DAC tile 2 (230) is fully powered up!
2024-05-09 17:18:11,398 - root - INFO - DAC tile 2 DAC block 0 is enabled!
2024-05-09 17:18:11,400 - root - INFO - DAC tile 2 DAC block 1 is NOT enabled!
2024-05-09 17:18:11,403 - root - INFO - DAC tile 2 DAC block 2 is NOT enabled!
2024-05-09 17:18:11,405 - root - INFO - DAC tile 2 DAC block 3 is NOT enabled!
2024-05-09 17